In [1]:
import json
import requests

In [2]:
#https://pypi.org/project/gamma-viewer/
from gamma_viewer import GammaViewer
from IPython.display import display

In [3]:
def submit_to_ars(m,ars_url='https://ars.transltr.io/ars/api',arax_url='https://arax.ncats.io/legacy'):
    submit_url=f'{ars_url}/submit'
    response = requests.post(submit_url,json=m)
    try:
        message_id = response.json()['pk']
    except:
        print('fail')
        message_id = None
    print(f'{arax_url}/?source=ARS&id={message_id}')
    return message_id

def retrieve_ars_results(mid,ars_url='https://ars.transltr.io/ars/api'):
    message_url = f'{ars_url}/messages/{mid}?trace=y'
    response = requests.get(message_url)
    j = response.json()
    print( j['status'] )
    results = {}
    for child in j['children']:
        print(child['status'])
        if child['status']  == 'Done':
            childmessage_id = child['message']
            child_url = f'{ars_url}/messages/{childmessage_id}'
            try:
                child_response = requests.get(child_url).json()
                nresults = len(child_response['fields']['data']['message']['results'])
                if nresults > 0:
                    results[child['actor']['agent']] = {'message':child_response['fields']['data']['message']}
            except Exception as e:
                nresults=0
                child['status'] = 'ARS Error'
        elif child['status'] == 'Error':
            nresults=0
            childmessage_id = child['message']
            child_url = f'{ars_url}/messages/{childmessage_id}'
            try:
                child_response = requests.get(child_url).json()
                results[child['actor']['agent']] = {'message':child_response['fields']['data']['message']}
            except Exception as e:
                print(e)
                child['status'] = 'ARS Error'
        else:
            nresults = 0
        print( child['status'], child['actor']['agent'],nresults )
    return results

In [4]:
def submit_to_devars(m):
    return submit_to_ars(m,ars_url='https://ars-dev.transltr.io/ars/api',arax_url='https://arax.ncats.io')

def retrieve_devars_results(m):
    return retrieve_ars_results(m,ars_url='https://ars-dev.transltr.io/ars/api')

In [5]:
def printjson(j):
    print(json.dumps(j,indent=4))

def print_json(j):
    printjson(j)

In [6]:
def name_lookup(text):
    url= f'https://name-resolution-sri.renci.org/lookup?string={text}&offset=0&limit=10'
    response = requests.post(url)
    printjson(response.json())

In [7]:
def one_hop_message(subject_category,object_category,predicate=None,subject_id = None, object_id = None):
    query_graph = {
    "nodes": {
        'a':{
            "category": subject_category
        },
        'b':{
            "category": object_category
        }
    },
    "edges": {
        'ab':{
            "subject": "a",
            "object": "b",
            #"predicate": predicate
        }
    }
    }
    if predicate is not None:
        query_graph['edges']['ab']['predicate'] = predicate
    if subject_id is not None:
        query_graph['nodes']['a']['id'] = subject_id
    if object_id is not None:
        query_graph['nodes']['b']['id'] = object_id
    message = {"message": {"query_graph": query_graph},'knowledge_graph':{"nodes": [], "edges": [],}, 'results':[]}
    return message

# DILI Drug Repurposing

In [15]:
dqg={
  "nodes": {
    "n0": {
      "name": "drug-induced liver injury",
      "ids": ["MONDO:0005359"]
    },
    "n1": {
      "categories": [
        "biolink:DiseaseOrPhenotypicFeature"
      ],
      "name": "Disease Or Phenotypic Feature"
    },
    "n2": {
      "categories": [
        "biolink:Gene"
      ],
      "name": "Gene"
    },
    "n3": {
      "categories": [
        "biolink:ChemicalSubstance"
      ],
      "name": "Chemical Substance"
    }
  },
  "edges": {
    "e0": {
      "subject": "n0",
      "object": "n1",
      "predicates": [
        "biolink:correlated_with"
      ]
    },
    "e1": {
      "subject": "n1",
      "object": "n2",
      "predicates": [
        "biolink:condition_associated_with_gene"
      ]
    },
    "e2": {
      "subject": "n2",
      "object": "n3",
      "predicates": [
        "biolink:related_to"
      ]
    }
  }
}

threehopquery = {'message': {'query_graph':dqg}}

In [16]:
threehopresult = submit_to_devars(threehopquery)

https://arax.ncats.io/?source=ARS&id=0445a10f-05b6-4bf3-82a5-586a55655e1c


In [18]:
hop1={
  "nodes": {
    "n0": {
      "name": "drug-induced liver injury",
      "ids": ["MONDO:0005359"]
    },
    "n1": {
      "categories": [
        "biolink:DiseaseOrPhenotypicFeature"
      ],
      "name": "Disease Or Phenotypic Feature"
    }
  },
  "edges": {
    "e0": {
      "subject": "n0",
      "object": "n1",
      "predicates": [
        "biolink:correlated_with"
      ]
    }
  }
}

onehopquery = {'message': {'query_graph':hop1}}

In [19]:
onehopresult = submit_to_devars(onehopquery)

https://arax.ncats.io/?source=ARS&id=28390bb4-3831-4cb7-b7bf-df08a80f2585


In [20]:
hop1allpred={
  "nodes": {
    "n0": {
      "name": "drug-induced liver injury",
      "ids": ["MONDO:0005359"]
    },
    "n1": {
      "categories": [
        "biolink:DiseaseOrPhenotypicFeature"
      ],
      "name": "Disease Or Phenotypic Feature"
    }
  },
  "edges": {
    "e0": {
      "subject": "n0",
      "object": "n1",
      "predicates": [
        "biolink:related_to"
      ]
    }
  }
}

onehopallpredquery = {'message': {'query_graph':hop1allpred}}

In [21]:
onehopallpredresult = submit_to_devars(onehopallpredquery)

https://arax.ncats.io/?source=ARS&id=5805892a-6365-44ab-8bdf-177c6bc5a555


In [22]:
hop1snomed={
  "nodes": {
    "n0": {
      "name": "drug-induced liver injury",
      "ids": ["SNOMEDCT:197358007"]
    },
    "n1": {
      "categories": [
        "biolink:DiseaseOrPhenotypicFeature"
      ],
      "name": "Disease Or Phenotypic Feature"
    }
  },
  "edges": {
    "e0": {
      "subject": "n0",
      "object": "n1",
      "predicates": [
        "biolink:correlated_with"
      ]
    }
  }
}

onehopsnomedquery = {'message': {'query_graph':hop1snomed}}

In [23]:
onehopsnomedresult = submit_to_devars(onehopsnomedquery)

https://arax.ncats.io/?source=ARS&id=7c8d4771-cbdb-4e71-84bc-33fb65e068cb


In [24]:
twohopsnomed={
  "nodes": {
    "n0": {
      "name": "drug-induced liver injury",
      "ids": ["SNOMEDCT:197358007"]
    },
    "n1": {
      "categories": [
        "biolink:DiseaseOrPhenotypicFeature"
      ],
      "name": "Disease Or Phenotypic Feature"
    },
    "n2": {
      "categories": [
        "biolink:Gene"
      ],
      "name": "Gene"
    }
  },
  "edges": {
    "e0": {
      "subject": "n0",
      "object": "n1",
      "predicates": [
        "biolink:correlated_with"
      ]
    },
    "e1": {
      "subject": "n1",
      "object": "n2",
      "predicates": [
        "biolink:condition_associated_with_gene"
      ]
    }
  }
}

twohopsnomedquery = {'message': {'query_graph':twohopsnomed}}

In [25]:
twohopsnomedresult = submit_to_devars(twohopsnomedquery)

https://arax.ncats.io/?source=ARS&id=12cf506e-bb4b-425e-851e-7e91bc90e489
